In [1]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from os.path import isfile, join
import os
import ast
from stable_baselines import DQN
import sys
sys.path.append('..')
from modules import utils, constants
from multiprocessing import Process
from stable_baselines.common.vec_env import DummyVecEnv
from modules.env import LupusEnv
import warnings
warnings.filterwarnings('ignore')

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.i

In [2]:
# SEED = constants.SEED
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.set_random_seed(constants.SEED)
tf.compat.v1.set_random_seed(constants.SEED)
SEED

42

#### Reading the data

In [3]:
val_df = pd.read_csv('../data/val_set_constant.csv')
val_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,1
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,1,0,...,0,1,0,1,1,0,0,0,0,1


In [4]:
X_val = val_df.iloc[:, 0:-1]
y_val = val_df.iloc[:, -1]
X_val, y_val = np.array(X_val), np.array(y_val)
X_val.shape, y_val.shape

((5600, 23), (5600,))

#### start here

In [5]:
# best_f1, best_acc, best_roc_auc = -1, -1, -1
# perf_list = []
# count = 0
# model_name = 'dqn'
# seed = SEED
# prefix= 'dqn'
# folder = f'../models/logs/{model_name}/seed_{seed}_{steps}'    

In [6]:
# item = os.listdir(folder)[0]
# item

In [7]:
# if item.startswith(prefix):
#     path= join(folder, item)
#     print(path)
#     if (isfile(path)) & (path.endswith('.zip')):
#         val_env = DummyVecEnv([lambda: LupusEnv(X_val, y_val, random=False)])
#         val_env.seed(seed)
#         print(type(val_env))
#         model = utils.load_dqn(path, val_env)
#         print(type(model))

#### end here

In [8]:
# def get_val_metrics(model, validation_env):
#     val_df = pd.DataFrame()
#     try:
#         while True:
#             obs, done = validation_env.reset(), False
#             while not done:
#                 action, states = model.predict(obs, deterministic=True)
#                 obs, rew, done, info = validation_env.step(action)
#                 if done==True:
#                     val_df = val_df.append(info, ignore_index=True)

#     except StopIteration:
#         pass
#     acc, f1, roc_auc, = test(val_df['y_actual'], val_df['y_pred'])
#     min_path_length = val_df.episode_length.min()
#     average_path_length = val_df.episode_length.mean()
#     max_path_length = val_df.episode_length.max()
#     min_pathway = val_df[val_df.episode_length==min_path_length].trajectory.iloc[0]
#     max_pathway = val_df[val_df.episode_length==max_path_length].trajectory.iloc[0]
#     return acc, f1, roc_auc, min_path_length, average_path_length, max_path_length, min_pathway, max_pathway

In [9]:
def get_steps(filename, prefix):
    try:
        return int(filename[len(prefix)+1:][:-10])
    except Exception as e:
        print(f'Filename: {filename}')
        print(f'Exception: {e}')

In [10]:
# def check_pathway(pathway):
#     try:
#         pathway = ast.literal_eval(pathway)
#     except:
#         pass
#     return len(pathway), pathway[0] 

In [11]:
# get_steps('dqn_66000000_steps.zip', 'dqn')

In [12]:
def validate_model(model_name, seed, steps, X_val, y_val, prefix):
    best_f1, best_acc, best_roc_auc = -1, -1, -1
    perf_list = []
    count = 0
    
    folder = f'../models/logs/{model_name}/risk_factor/seed_{seed}_{steps}'    
    for item in os.listdir(folder):
#         print(item)
        count+=1
        if count%10 == 0:
            print(count)
        if item.startswith(prefix):
#             print('MODELS FOUND')
            path = join(folder, item)
#             print(path)
            if (isfile(path)) & (path.endswith('.zip')):
#                 print(f'ZIPPED MODELS FOUND at {path}')
#                 validation_env.observation_space = Box(0, 3, (constants.FEATURE_NUM,))
#                 print('LOADING MODEL ..')
                val_env = DummyVecEnv([lambda: LupusEnv(X_val, y_val, random=False)])
                val_env.seed(seed)
#                 print(f'ENVIRONMENT TYPE:{type(val_env)}')
                model = utils.load_dqn(path, val_env)
#                 print(type(model))
#                 print(f'MODEL TYPE: {type(model)}')
#                 print(f'MODEL LOADED: {model}')
                acc, f1, roc_auc, min_length, avg_length, max_length, min_path, max_path = utils.get_val_metrics(
                    model, val_env)
#                 print(f'ACCURACY FOR {path} IS {acc}')
                perf_dict = {'steps': get_steps(item, prefix), 'acc':acc, 'f1':f1, 'roc_auc':roc_auc, 
                             'min_path_length':min_length, 'avg_length':avg_length, 'max_length':max_length, 
                             'min_path':min_path, 'max_path':max_path} 
                perf_list.append(perf_dict)
                if acc > best_acc:
#                     print('SAVING BEST ACC MODEL')
                    best_acc = acc
                    model.save(f'{folder}/best_acc_model')
                if f1 > best_f1:
                    best_f1 = f1
                    model.save(f'{folder}/best_f1_model')
                if roc_auc > best_roc_auc:
                    best_roc_auc = roc_auc
                    model.save(f'{folder}/best_roc_auc_model')
                    
        val_df = pd.DataFrame.from_dict(perf_list) 
        val_df = val_df.sort_values(by=['steps'])
        val_df = val_df.reset_index(drop=True)
        val_df.to_csv(f'{folder}/validation_results.csv', index=False)
    return val_df          

In [13]:
# names = ['dqn', 'ddqn', 'dueling_dqn', 'dueling_ddqn', 'dqn_per', 'ddqn_per', 'dueling_dqn_per', 'dueling_ddqn_per']
# for name in names[1:]:
#     print(f'{name.upper()} STARTING!')
#     eval_df = validate_model(name, SEED, steps, X_val, y_val, name)
# # eval_df.head()

In [14]:
# models = ['dqn', 'ddqn', 'dueling_dqn', 'dueling_ddqn', 'dqn_per', 'ddqn_per', 'dueling_dqn_per', 
#'dueling_ddqn_per']
models = ['dqn_per']
steps =int(10e7)
procs=[]

In [15]:
for name in models:
    proc = Process(target=validate_model, args=(name, SEED, steps, X_val, y_val, name))
    procs.append(proc)
    proc.start()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.
10
20
30
40
50
60
70
80
90
100


In [16]:
for proc in procs:
    proc.join()
print('All jobs completed and terminated successfully')

All jobs completed and terminated successfully
